# Steps:

1. Custom CNN for Feature Extraction
2. Texture Analysis using GLCM
3. Saving Features to CSV
4. Training an ANN Model
5. Model Evaluation Function

In [1]:
# libs
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.feature import greycomatrix, greycoprops

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report

import os

import warnings
warnings.filterwarnings('ignore')

In [11]:
# Step 1: Custom CNN Model for Feature Extraction
def create_custom_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    model = models.Model(inputs, x)
    return model

In [4]:
# Step 2: Function to calculate GLCM properties
def calculate_glcm_features(image, distances, angles):
    properties = ['contrast', 'dissimilarity', 'homogeneity', 
                  'energy', 'correlation', 'ASM']
    glcm = greycomatrix(image, 
                        distances = distances, 
                        angles = angles, 
                        symmetric = True, 
                        normed = True)
    feature_vector = []
    
    for prop in properties:
        prop_values = greycoprops(glcm, prop)
        feature_vector.extend(prop_values.flatten())
    
    return feature_vector

In [5]:
# Step 3: Fetch CNN features, GLCM features, and save to CSV
def extract_features_and_save(images, labels, cnn_model, output_csv, distances, angles):
    feature_list = []
    for i, image in enumerate(images):
        print(f"Processing image {i+1}/{len(images)}")
        
        # CNN Features
        image_expanded = np.expand_dims(image, axis=0)  # Expand dimensions for the CNN model
        cnn_features = cnn_model.predict(image_expanded).flatten()
        
        # GLCM Features
        image_gray = np.mean(image, axis=-1).astype(np.uint8)  # Convert to grayscale if needed
        glcm_features = calculate_glcm_features(image_gray, distances, angles)
        
        # Combine CNN + GLCM features
        combined_features = np.concatenate([cnn_features, glcm_features])
        feature_list.append(np.concatenate([combined_features, [labels[i]]]))
    
    # Create a DataFrame and save to CSV
    df = pd.DataFrame(feature_list)
    df.to_csv(output_csv, index=False)
    print(f"Features saved to {output_csv}")

In [6]:
# Step 4: Create ANN model to train using extracted features
def create_ann_model(input_dim):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_dim=input_dim),
        layers.Dense(32, activation='relu'),
        #layers.Dense(1, activation='sigmoid')  # Adjust depending on the classification task (binary/multi-class)
        layers.Dense(3, activation='softmax')  # Adjust depending on the classification task (binary/multi-class)
    ])
    
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Adjust loss if multi-class
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Adjust loss if multi-class
    return model

In [16]:
# Step 5: Function to evaluate ANN model
def evaluate_model(model, X_test, y_test):
    #y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Binary thresholding for binary classification
    #print(classification_report(y_test, y_pred))
    y_pred = np.argmax(model.predict(X_test), axis=1)  # Get class with highest probability
    y_true = np.argmax(y_test, axis=1)  # Convert one-hot to labels for classification report
    print(classification_report(y_true, y_pred))

In [8]:
# Step 6: Load your dataset
def load_dataset_from_directory(directory, img_size=(128, 128), batch_size=32):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='int',  # 'int' for integer labels, change to 'categorical' if needed
        image_size=img_size,  # Resize all images to the target size
        batch_size=batch_size,
        shuffle=True
    )
    return dataset

In [17]:

if __name__ == "__main__":
    # Step 6: Load your dataset
    # Load training data
    train_directory = os.path.join('..', 'Dataset', 'data', 'train')
    train_dataset = load_dataset_from_directory(train_directory)
    
    # Extract images and labels from the dataset
    images = []
    labels = []
    
    for batch in train_dataset:
        imgs, lbls = batch
        images.append(imgs.numpy())  # Convert to numpy arrays
        labels.append(lbls.numpy())
    
    # Convert lists to numpy arrays
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    print(f"Loaded {images.shape[0]} images of shape {images.shape[1:]}")
    print(f"Loaded {labels.shape[0]} labels")

    # Step 7: CNN Model for feature extraction
    cnn_model = create_custom_cnn(input_shape=(128, 128, 3))
    cnn_model.summary()
    
    # Step 8: GLCM parameters
    distances = [1, 3, 5, 3, 1, 3, 5]
    angles = [0, 0, 0, np.pi/4, np.pi/2, np.pi/2, np.pi/2]

    # Step 9: Extract features and save to CSV
    output_csv = 'features.csv'
    #extract_features_and_save(images, labels, cnn_model, output_csv, distances, angles)

    # Step 10: Load CSV and prepare for ANN training
    data = pd.read_csv(output_csv)
    X = data.iloc[:, :-1].values  # All columns except the last one (features)
    y = data.iloc[:, -1].values  # Last column (labels)
    
    # One-hot encode labels for multi-class classification
    y = tf.keras.utils.to_categorical(y, num_classes=3)
    
    # Preprocessing: Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"X_train.shape[1] : {X_train.shape[1]}")
    
    # Step 11: Create and train ANN
    ann_model = create_ann_model(input_dim=X_train.shape[1])
    ann_model.summary()
    
    ann_model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.1)

    # Step 12: Evaluate the model
    evaluate_model(ann_model, X_test, y_test)

Found 1206 files belonging to 3 classes.
Loaded 1206 images of shape (128, 128, 3)
Loaded 1206 labels
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 64, 64, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 32, 32, 64)        0         
 ng2D)                                                           
                       

55/55 [==============================] - 0s 3ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0601 - val_accuracy: 0.9897
Epoch 42/50
55/55 [==============================] - 0s 3ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0606 - val_accuracy: 0.9897
Epoch 43/50
55/55 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0590 - val_accuracy: 0.9897
Epoch 44/50
55/55 [==============================] - 0s 3ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0604 - val_accuracy: 0.9897
Epoch 45/50
55/55 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0607 - val_accuracy: 0.9794
Epoch 46/50
55/55 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0592 - val_accuracy: 0.9897
Epoch 47/50
55/55 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0590 - val_accuracy: 0.9897
Epoch 48/50
55/55 [======